# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.121426e+02     1.478792e+00
 * time: 0.6858799457550049
     1     1.057181e+01     8.328493e-01
 * time: 1.9896130561828613
     2    -1.180864e+01     9.618209e-01
 * time: 2.6677448749542236
     3    -3.396182e+01     7.698895e-01
 * time: 3.6044809818267822
     4    -4.753466e+01     5.391099e-01
 * time: 4.565170049667358
     5    -5.684615e+01     2.305172e-01
 * time: 5.536135911941528
     6    -5.983675e+01     1.135475e-01
 * time: 6.178966045379639
     7    -6.094484e+01     5.079858e-02
 * time: 6.82000994682312
     8    -6.135153e+01     8.343486e-02
 * time: 7.474202871322632
     9    -6.167919e+01     3.509831e-02
 * time: 8.116654872894287
    10    -6.188547e+01     2.845213e-02
 * time: 8.754173040390015
    11    -6.203881e+01     2.019254e-02
 * time: 9.396610975265503
    12    -6.209457e+01     1.930068e-02
 * time: 10.05132508277893
    13    -6.214594e+01     1.288291e-02
 * time: 10.686218976974487
   

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671060
    AtomicLocal         -18.8557689
    AtomicNonlocal      14.8522667
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485367 
    Xc                  -19.3336818

    total               -62.261666449616
